In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # typically done at the start of the script
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.metrics import log_loss



from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.decomposition import PCA
# from sklearn.neural_network import MLPClassifier
# import seaborn as sns # for visualiation
# from sklearn.datasets import make_blobs
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import confusion_matrix






In [2]:
## wrangled data
data = pd.read_csv('./data/funding_and_scores.csv')
data.head()

,Unnamed: 0,State,year,Number.of.districts,Minimum,Maximum,Mean,score
0,1,Alabama,1995,127,5691657.6,405581866.2,4.101663e+07,0.701875
1,2,Alabama,2000,128,7786867.2,551756732.8,5.373821e+07,0.695625
2,3,Alabama,2005,131,6607702.2,661439354.4,5.463843e+07,0.704583
3,4,Alabama,2010,132,6317688.3,631356264.0,6.290127e+07,0.676250
4,5,Alabama,2014,136,5017000.0,580513000.0,5.626182e+07,0.673333


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 8 columns):
Unnamed: 0             260 non-null int64
State                  260 non-null object
year                   260 non-null int64
Number.of.districts    260 non-null object
Minimum                260 non-null float64
Maximum                260 non-null float64
Mean                   260 non-null float64
score                  260 non-null float64
dtypes: float64(4), int64(2), object(2)
memory usage: 16.3+ KB


In [4]:
# numeric = data.select_dtypes(include=['int64', 'float64'])

In [5]:
# Standardize features
def standardize(df, numeric_only=True):
    numeric = df.select_dtypes(include=['int64', 'float64'])
    
    # subtracy mean and divide by std
    df[numeric.columns] = (numeric - numeric.mean()) / numeric.std()
    
    return df

def pre_process_data(df):
    print("Input shape:\t{}".format(df.shape))
        

    df = standardize(df)
    print("After standardization {}".format(df.shape))
        
    # create dummy variables for categoricals
    df = pd.get_dummies(df)
    print("After converting categoricals:\t{}".format(df.shape))
    

#     # match test set and training set columns
#     if enforce_cols is not None:
#         to_drop = np.setdiff1d(df.columns, enforce_cols)
#         to_add = np.setdiff1d(enforce_cols, df.columns)

#         df.drop(to_drop, axis=1, inplace=True)
#         df = df.assign(**{c: 0 for c in to_add})
    
    df.fillna(0, inplace=True)
    
    return df

In [6]:
# test = data[data['year'] == 2014]


In [8]:
data_label = data.score
data_label

0      0.701875
1      0.695625
2      0.704583
3      0.676250
4      0.673333
5      0.646250
6      0.640000
7      0.636250
8      0.630417
9      0.622500
10     0.653750
11     0.655000
12     0.648333
13     0.641250
14     0.646667
15     0.697500
16     0.695000
17     0.712083
18     0.705000
19     0.703333
20     0.628750
21     0.634375
22     0.633333
23     0.630417
24     0.621667
25     0.671250
26     0.675625
27     0.695833
28     0.707917
29     0.723333
         ...   
230    0.628750
231    0.635625
232    0.639167
233    0.640833
234    0.647500
235    0.626875
236    0.631875
237    0.635417
238    0.631667
239    0.638750
240    0.648750
241    0.658750
242    0.654167
243    0.650000
244    0.623333
245    0.645000
246    0.649375
247    0.643333
248    0.630000
249    0.625417
250    0.726875
251    0.737500
252    0.735417
253    0.736250
254    0.737917
255    0.680000
256    0.682500
257    0.683333
258    0.705000
259    0.723750
Name: score, Length: 260

In [10]:
data_feature = pre_process_data(data.drop('score', axis=1))


Input shape:	(260, 7)
After standardization (260, 7)
After converting categoricals:	(260, 240)


In [11]:
data_feature.head()

,Unnamed: 0,year,Minimum,Maximum,Mean,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,...,Number.of.districts_823,Number.of.districts_847,Number.of.districts_89,Number.of.districts_892,Number.of.districts_916,Number.of.districts_94,Number.of.districts_97,Number.of.districts_98,Number.of.districts_983,Number.of.districts_995
0,-1.722081,-1.439648,-0.148117,-0.362987,-0.202067,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-1.708783,-0.705134,-0.141419,-0.328042,-0.161824,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-1.695485,0.029381,-0.145189,-0.301822,-0.158976,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-1.682187,0.763895,-0.146116,-0.309013,-0.132838,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-1.668889,1.351506,-0.150273,-0.321168,-0.153841,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
c = pre_process_data(data_feature)

Input shape:	(260, 240)
After standardization (260, 240)
After converting categoricals:	(260, 240)


In [13]:
## Split data to training and testing 
train_features, test_features, train_outcome, test_outcome = train_test_split(data_feature, data_label, test_size = 0.3, random_state = 11)


In [20]:
## Split training data to training and validation
train_small_features, validation_features, train_small_outcome, validation_outcome = train_test_split(train_features, train_outcome, test_size = 0.2, random_state = 11)
train_small_features.head()

,Unnamed: 0,year,Minimum,Maximum,Mean,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,...,Number.of.districts_823,Number.of.districts_847,Number.of.districts_89,Number.of.districts_892,Number.of.districts_916,Number.of.districts_94,Number.of.districts_97,Number.of.districts_98,Number.of.districts_983,Number.of.districts_995
119,-0.139628,1.351506,-0.166298,-0.298842,-0.263228,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
205,1.003993,-1.439648,-0.165660,-0.421381,-0.312550,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107,-0.299203,0.029381,-0.166311,-0.158623,-0.211023,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
167,0.498672,0.029381,-0.164668,-0.153899,-0.131060,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
228,1.309845,0.763895,-0.164690,-0.329240,-0.189251,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
### logistic regression ###
param_grid = {' }

In [18]:
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=10, scoring=make_scorer(log_loss))


In [19]:
grid_search.fit(train_small_features, train_small_outcome)

ValueError: Unknown label type: 'continuous'

,Unnamed: 0,year,Minimum,Maximum,Mean,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,...,Number.of.districts_823,Number.of.districts_847,Number.of.districts_89,Number.of.districts_892,Number.of.districts_916,Number.of.districts_94,Number.of.districts_97,Number.of.districts_98,Number.of.districts_983,Number.of.districts_995
119,-0.139628,1.351506,-0.166298,-0.298842,-0.263228,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
205,1.003993,-1.439648,-0.165660,-0.421381,-0.312550,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107,-0.299203,0.029381,-0.166311,-0.158623,-0.211023,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
167,0.498672,0.029381,-0.164668,-0.153899,-0.131060,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
228,1.309845,0.763895,-0.164690,-0.329240,-0.189251,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34,-1.269951,1.351506,-0.165269,-0.339047,-0.164226,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176,0.618353,-0.705134,-0.166311,-0.250343,-0.247576,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,1.589101,1.351506,-0.157376,-0.379467,-0.156082,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101,-0.378991,-0.705134,-0.060657,0.000906,1.001080,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
203,0.977397,0.763895,-0.163017,-0.284932,-0.040999,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
knc = KNeighborsClassifier()

param_grid_knc = {'kneighborsclassifier__n_neighbors': np.arange(1,10)}

rfc = RandomForestClassifier()

param_grid_rfc = {'randomforestclassifier__n_estimators': np.arange(1,10)}

dtc = DecisionTreeClassifier()

param_grid_dtc = {'decisiontreeclassifier__max_depth': np.arange(1,10)}

nn = MLPClassifier() 

param_grid_clf = {'mlpclassifier__hidden_layer_sizes': np.arange(1,12),

                  'mlpclassifier__activation': ['identity', 'logistic', 'tanh', 'relu']}

nn.get_params().keys()

In [ ]:
def run_model(model, param_grid, xtrain, ytrain, do_pca = False):
    if(do_pca == True):
        pca = PCA(n_components = 10)
        scaler = MinMaxScaler()
        pipe = make_pipeline(pca, model)
        grid = GridSearchCV(pipe,param_grid)
        grid.fit(xtrain, ytrain)
        grid.best_params_
        accuracy = grid.score(xtrain, ytrain)
        print(f"In-sample accuracy: {accuracy:0.2%}")
        return(grid)
    scaler = MinMaxScaler()
    pipe = make_pipeline(model)
    grid = GridSearchCV(pipe,param_grid)
    grid.fit(xtrain, ytrain)
    grid.best_params_
    accuracy = grid.score(xtrain, ytrain)
    print(f"In-sample accuracy: {accuracy:0.2%}")
    return(grid)

In [ ]:
reg = linear_model.Lasso()

param_grid_reg = {'lasso__alpha': np.arange(0.05,0.2),
                  'lasso__copy_X': [True, False],
                 'lasso__fit_intercept': [True, False],
                 'lasso__max_iter': np.arange(500, 2000),
                 'lasso__normalize': [True, False],
                 'lasso__positive': [True, False],
                 'lasso__precompute': [True, False],
                 'lasso__selecton': ['cyclic'],
                 'lasso__warm_start': [True, False]}

In [ ]:
run_model(reg, param_grid_reg, train_small_features, train_small_outcome, True)

In [ ]:
from sklearn.linear_model import ElasticNetCV
regr = ElasticNetCV()
param_grid_regr = {'elasticnetcv__copy_X': [True, False],
                 'elasticnetcv__fit_intercept': [True, False],
                 'elasticnetcv__max_iter': np.arange(500, 2000),
                 'elasticnetcv__normalize': [True, False],
                 'elasticnetcv__cv': np.arange(5,10),
                  'elasticnetcv__random_state': np.arange(0,1),
                 'elasticnetcv__n_jobs': np.arange(1,5),
                 'elasticnetcv__selecton': ['cyclic']}

In [ ]:
run_model(regr, param_grid_regr, train_small_features, train_small_outcome, do_pca = False)

In [ ]:
est